In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

# package for simple neural network

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [8]:
# Load the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train / 255
x_test = x_test / 255

print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [9]:
# Build the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(10, activation='softmax'))


In [10]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 1, 1, 128)       

In [12]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Checkpoint
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
model.fit(x_train.reshape(-1, 28, 28, 1), y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping, checkpoint])

Epoch 1/100
1500/1500 [==============================] - 25s 16ms/step - loss: 0.1492 - accuracy: 0.9558 - val_loss: 0.0682 - val_accuracy: 0.9798
Epoch 2/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0761 - accuracy: 0.9779 - val_loss: 0.0579 - val_accuracy: 0.9830
Epoch 3/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0567 - accuracy: 0.9825 - val_loss: 0.0502 - val_accuracy: 0.9853
Epoch 4/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0453 - accuracy: 0.9864 - val_loss: 0.0473 - val_accuracy: 0.9859
Epoch 5/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0360 - accuracy: 0.9889 - val_loss: 0.0499 - val_accuracy: 0.9852
Epoch 6/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0283 - accuracy: 0.9915 - val_loss: 0.0538 - val_accuracy: 0.9851
Epoch 7/100
1500/1500 [==============================] - 24s 16ms/step - loss: 0.0284 - accuracy: 0.9912 - val_loss: 0

In [13]:
# Evaluate the model
model.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0536 - accuracy: 0.9877


[0.053570982068777084, 0.9876999855041504]

In [14]:
# ROC_AUC score
from sklearn.metrics import roc_auc_score
y_pred = model.predict(x_test.reshape(-1, 28, 28, 1))
roc_auc_score(y_test, y_pred, multi_class='ovr')

0.9997578790625387